In [ ]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import numpy as np
from keras.layers import InputSpec
import keras.backend as K
from keras import regularizers
from keras.layers import ZeroPadding2D, AveragePooling2D,BatchNormalization,Activation
from keras.models import Model
from keras.layers import Merge, Input, Lambda, Layer, add
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras_ode.layers import HamiltonianConv2D, ChannelZeroPadding

(x_train, y_train) , (x_test, y_test) = cifar10.load_data()

num_classes = 10

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [ ]:
1+1

In [2]:
# https://github.com/raghakot/keras-resnet/blob/master/cifar10.py
# subtract mean and normalize
mean_image = np.mean(x_train, axis=0)
x_train -= mean_image
x_test -= mean_image
x_train /= 128.
x_test /= 128.

In [5]:
img_input = Input(shape=x_train.shape[1:])
x = ZeroPadding2D(padding=(3, 3), name='conv1_pad')(img_input)
x = Conv2D(16, (7, 7), strides=(1, 1), kernel_regularizer=regularizers.l2(.0001),
           kernel_initializer='he_normal', padding='valid')(x)
x = BatchNormalization(axis=-1)(x)
x = Activation('relu')(x)
    
x = HamiltonianConv2D(filters=16, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = AveragePooling2D((2, 2))(x)
x = ChannelZeroPadding(padding=4)(x)

x = HamiltonianConv2D(filters=64, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = ChannelZeroPadding(padding=2)(x)

x = HamiltonianConv2D(filters=128, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = ChannelZeroPadding(padding=2)(x)

x = HamiltonianConv2D(filters=256, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = AveragePooling2D((2, 2))(x)
x = ChannelZeroPadding(padding=2)(x)

x = HamiltonianConv2D(filters=512, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = AveragePooling2D((2, 2))(x)
x = ChannelZeroPadding(padding=2)(x)

x = HamiltonianConv2D(filters=1024, h=.15, unroll_length=4, kernel_regularizer=regularizers.l2(.0001),
                    kernel_initializer='he_normal', kernel_size=(3,3),padding='same', activation='relu')(x)

x = AveragePooling2D((4, 4))(x)

x = Flatten()(x)
out = Dense(10, kernel_regularizer=regularizers.l2(.0001), activation='softmax')(x)
model = Model(img_input, out)

built
built
built
built
built
built


In [6]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.1, momentum=0.9),
              metrics=['accuracy'])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 38, 38, 3)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 16)        2368      
_________________________________________________________________
bn_conv1 (BatchNormalization (None, 32, 32, 16)        64        
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 16)        0         
_________________________________________________________________
hamiltonian_conv_7 (Hamilton (None, 32, 32, 16)        592       
_________________________________________________________________
average_pooling2d_5 (Average (None, 16, 16, 16)        0         
__________

In [8]:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

datagen.fit(x_train)


In [9]:
batch_size=32

In [10]:
steps_in_epoch = (x_train.shape[0] // batch_size)

In [11]:
training_iterations = 64000*8

In [12]:
epochs = training_iterations / steps_in_epoch

In [13]:
epochs

327

In [14]:
drops = [(training_iterations/3) / steps_in_epoch,
         (2*training_iterations/3 )/ steps_in_epoch]

In [15]:
drops

[109, 218]

In [16]:
from keras.callbacks import ReduceLROnPlateau, CSVLogger, EarlyStopping,LearningRateScheduler
    
def step_decay(epoch):
    initial_lrate = 0.1
    drop = 10.
    
    
    if epoch < drops[0]:
        power= 0
    elif epoch < drops[1]:
        power = 1
    else:
        power = 2
    
    
    lrate = initial_lrate / np.power(drop, power)
    print ('epoch:' + str(epoch) + ' learning rate:' + str(lrate)) # added
    return lrate
        
        

lr_scheduler = LearningRateScheduler(schedule=step_decay)
csv_logger = CSVLogger('training_result.csv')

In [583]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=steps_in_epoch,
                    validation_data=(x_test, y_test),
                    epochs=epochs, verbose=1, max_q_size=10,
                    callbacks=[lr_scheduler, csv_logger])

/bighd/venvs/base/lib/python2.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, validation_data=(array([[[..., steps_per_epoch=1562, epochs=327, callbacks=[<keras.ca..., max_queue_size=10)`
  


epoch:0 learning rate:0.1
Epoch 1/327
1562/1562 [==============================] - 284s 182ms/step - loss: 1.8719 - acc: 0.3557 - val_loss: 1.5520 - val_acc: 0.4827
epoch:1 learning rate:0.1
Epoch 2/327
1562/1562 [==============================] - 282s 181ms/step - loss: 1.5287 - acc: 0.4948 - val_loss: 1.3262 - val_acc: 0.5790
epoch:2 learning rate:0.1
Epoch 3/327
1562/1562 [==============================] - 281s 180ms/step - loss: 1.3731 - acc: 0.5615 - val_loss: 1.3171 - val_acc: 0.6005
epoch:3 learning rate:0.1
Epoch 4/327
1562/1562 [==============================] - 280s 179ms/step - loss: 1.2890 - acc: 0.6038 - val_loss: 1.2283 - val_acc: 0.6254
epoch:4 learning rate:0.1
Epoch 5/327
1562/1562 [==============================] - 280s 179ms/step - loss: 1.2245 - acc: 0.6378 - val_loss: 1.2159 - val_acc: 0.6514
epoch:5 learning rate:0.1
Epoch 6/327
1562/1562 [==============================] - 280s 179ms/step - loss: 1.1836 - acc: 0.6593 - val_loss: 1.1436 - val_acc: 0.6818
epoch:6 le

1562/1562 [==============================] - 272s 174ms/step - loss: 0.9951 - acc: 0.7944 - val_loss: 1.0296 - val_acc: 0.7915
epoch:50 learning rate:0.1
Epoch 51/327
1562/1562 [==============================] - 272s 174ms/step - loss: 0.9939 - acc: 0.7959 - val_loss: 0.9812 - val_acc: 0.8002
epoch:51 learning rate:0.1
Epoch 52/327
1562/1562 [==============================] - 272s 174ms/step - loss: 1.0030 - acc: 0.7932 - val_loss: 1.1010 - val_acc: 0.7506
epoch:52 learning rate:0.1
Epoch 53/327
1562/1562 [==============================] - 272s 174ms/step - loss: 0.9825 - acc: 0.7997 - val_loss: 1.0229 - val_acc: 0.7873
epoch:53 learning rate:0.1
Epoch 54/327
1562/1562 [==============================] - 272s 174ms/step - loss: 0.9905 - acc: 0.7959 - val_loss: 1.0274 - val_acc: 0.7880
epoch:54 learning rate:0.1
Epoch 55/327
1562/1562 [==============================] - 272s 174ms/step - loss: 0.9925 - acc: 0.7983 - val_loss: 1.1525 - val_acc: 0.7532
epoch:55 learning rate:0.1
Epoch 56/32

1562/1562 [==============================] - 270s 173ms/step - loss: 0.4079 - acc: 0.9412 - val_loss: 0.5682 - val_acc: 0.8922
epoch:147 learning rate:0.01
Epoch 148/327
1562/1562 [==============================] - 281s 180ms/step - loss: 0.4111 - acc: 0.9409 - val_loss: 0.5854 - val_acc: 0.8896
epoch:148 learning rate:0.01
Epoch 149/327
1562/1562 [==============================] - 281s 180ms/step - loss: 0.4130 - acc: 0.9400 - val_loss: 0.5807 - val_acc: 0.8869
epoch:149 learning rate:0.01
Epoch 150/327
1562/1562 [==============================] - 281s 180ms/step - loss: 0.4068 - acc: 0.9419 - val_loss: 0.5889 - val_acc: 0.8867
epoch:150 learning rate:0.01
Epoch 151/327
1562/1562 [==============================] - 281s 180ms/step - loss: 0.4111 - acc: 0.9411 - val_loss: 0.5820 - val_acc: 0.8907
epoch:151 learning rate:0.01
Epoch 152/327
1562/1562 [==============================] - 281s 180ms/step - loss: 0.4098 - acc: 0.9407 - val_loss: 0.5869 - val_acc: 0.8901
epoch:152 learning rate

1562/1562 [==============================] - 272s 174ms/step - loss: 0.2535 - acc: 0.9957 - val_loss: 0.5573 - val_acc: 0.9117
epoch:243 learning rate:0.001
Epoch 244/327
1562/1562 [==============================] - 272s 174ms/step - loss: 0.2530 - acc: 0.9952 - val_loss: 0.5491 - val_acc: 0.9117
epoch:244 learning rate:0.001
Epoch 245/327
1562/1562 [==============================] - 273s 174ms/step - loss: 0.2524 - acc: 0.9956 - val_loss: 0.5482 - val_acc: 0.9101
epoch:245 learning rate:0.001
Epoch 246/327
1562/1562 [==============================] - 273s 175ms/step - loss: 0.2497 - acc: 0.9961 - val_loss: 0.5475 - val_acc: 0.9122
epoch:246 learning rate:0.001
Epoch 247/327
1562/1562 [==============================] - 273s 175ms/step - loss: 0.2490 - acc: 0.9955 - val_loss: 0.5469 - val_acc: 0.9120
epoch:247 learning rate:0.001
Epoch 248/327
1562/1562 [==============================] - 273s 174ms/step - loss: 0.2485 - acc: 0.9956 - val_loss: 0.5512 - val_acc: 0.9105
epoch:248 learning

1562/1562 [==============================] - 273s 175ms/step - loss: 0.2132 - acc: 0.9971 - val_loss: 0.5231 - val_acc: 0.9098
epoch:291 learning rate:0.001
Epoch 292/327
1562/1562 [==============================] - 273s 175ms/step - loss: 0.2134 - acc: 0.9964 - val_loss: 0.5261 - val_acc: 0.9088
epoch:292 learning rate:0.001
Epoch 293/327
1562/1562 [==============================] - 273s 175ms/step - loss: 0.2124 - acc: 0.9968 - val_loss: 0.5322 - val_acc: 0.9063
epoch:293 learning rate:0.001
Epoch 294/327
1562/1562 [==============================] - 273s 175ms/step - loss: 0.2110 - acc: 0.9970 - val_loss: 0.5364 - val_acc: 0.9050
epoch:294 learning rate:0.001
Epoch 295/327
1562/1562 [==============================] - 275s 176ms/step - loss: 0.2109 - acc: 0.9968 - val_loss: 0.5309 - val_acc: 0.9065
epoch:295 learning rate:0.001
Epoch 296/327
1562/1562 [==============================] - 276s 177ms/step - loss: 0.2106 - acc: 0.9966 - val_loss: 0.5175 - val_acc: 0.9103
epoch:296 learning